In [13]:
# show figures inline in the notebook
# %matplotlib qt
%matplotlib inline               

import matplotlib.pyplot as plt  # Import library for direct plotting functions
import numpy as np               # Import Numerical Python
import scipy.constants as const
from IPython.core.display import display, HTML #Import HTML for formatting output

# NOTE: Uncomment following lines ONLY if you are not using installation via pip
# import sys, os
# rootDir = '/path/to/arc/directory' # e.g. '/Users/Username/Desktop/ARC-Alkali-Rydberg-Calculator'
# sys.path.insert(0,rootDir)

from arc import *                 #Import ARC (Alkali Rydberg Calculator)

cs=Caesium() #Cesium has 55 electrons w/ valence 7S 
rb=Rubidium() #Rubidium has 37 electrons w/ valences 6S

In [14]:
#Declare atom variables
cs_gnd=9
rb_gnd=12
E_TO_COULOMB = 1.60217646*10**-19
A0_TO_METER = 5.29*10**-11
TOTAL_STATES = 100
POT = 20000000 #20MHz
POWER = 0.005
NA = .5 #Typical numerical aperture

atom = rb
atom_gnd = rb_gnd
name = "Caesium"
TEMP = 1

In [15]:
#Electron state generator for use with the ARC python package
#Given a Z, this class generates the list of occupied electron states for in input to the ARC energy level calculator

class State:
    
    #Definition of the principal QN, angular QN, magnetic QN, and spin QN
    def __init__(self,n,l,j,mj):
        self.n = n
        self.l = int(l)
        self.m = np.linspace(0,l,num= l+1)
        
        if j:
            self.j = j
        else:
            if self.l == 0:
                self.j = [.5]
            else:
                self.j = np.linspace(np.absolute(l-.5),l+.5,num=2)
        if mj:
            self.mj = mj
        
        else:
            self.mj = [[jj,None] for jj in self.j]

            for jj in self.mj:
                jj[1] = np.linspace(-jj[0],jj[0],num=(2*jj[0]+1))
            
class Transition:
    
    def __init__(self,s1,s2,atom,j):
        if not j:
            j = s2.j
        self.s1 = s1
        self.s2 = s2
        self.wvln = atom.getTransitionWavelength(s1.n,s1.l,s1.j,s2.n,s2.l,j)
        self.time = atom.getStateLifetime(s2.n,s2.l,j,0)
        self.rate = atom.getTransitionRate(s1.n,s1.l,s1.j,s2.n,s2.l,j,0)
        if atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,1) != 0:
            self.dipole = atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,1)*E_TO_COULOMB*A0_TO_METER
        elif atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,-1) != 0:
            self.dipole = atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,-1)*E_TO_COULOMB*A0_TO_METER
        elif atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,0) != 0:
            self.dipole = atom.getDipoleMatrixElement(s1.n,s1.l,s1.j,s1.mj,s2.n,s2.l,j,j,0)*E_TO_COULOMB*A0_TO_METER
        else: 
            self.dipole = 0.0
#Electron state generating class
class ElectronStates:

    def aufbau(self,n,l,e_list):

        e_list.append(State(n,l,None,None))
        self.ec -= 2*l+1
        
        #Conditionally recurses or returns remaining electron counts
        if l != 0:
            return self.aufbau(n+1,l-1,e_list)
        else:
            return self.ec
                
    
    #Class constructor for electron state generating class
    def __init__(self,Z,e_list):

        self.occ_n = []
        self.Z = Z
        self.ec = Z
        n=1
        l=0
         
        while self.ec > 0:

            self.aufbau(n,l,e_list)
            
            if (n+l)%2 == 0 and n < 5:
                l+=1
            else:
                n+=1




In [25]:
#TRANSITION INFO

trans_dict={}
pqn = {}


def calcPower(elemSum,wvln):
    
    return ((POT*const.epsilon_0*((wvln*const.Planck*const.c/NA)**2))/(elemSum*const.pi))

def calcScatterRate(wvln,elemSum):

    return ((4*const.pi**2*POWER*elemSum*NA**2)/(3*(wvln**2)*(const.epsilon_0**2)*(const.hbar**3)*(const.c**3)))

def calcPotDetuningSum(trans,wvln):

    return (trans.wvln*(trans.dipole**2.)/(1-(trans.wvln/wvln)**2))

def calcScatterDetuningSum(trans,wvln):
    return ((trans.dipole**4.)/(trans.wvln*((1-(trans.wvln/wvln)**2)**2)))


def createSpectrum(ground,wavelengths,states,atom):

    
    power = []
    scatter = []
    dp = []
    potElemSum = 0
    scatterElemSum = 0
    for wvln in wavelengths:
        potElemSum = 0
        scatterElemSum = 0
        for st in states:
            if not (ground.n < st.n and ground.l==st.l) and not ground.n > st.n \
            and not (ground.n == st.n and ground.l >= st.l):
#             if not ground.n > st.n and not (ground.n == st.n and ground.l >= st.l): 
#                 print(ground.n,ground.l,ground.j)
                for j in st.j:
#                     print(st.n,st.l,j)
                    trans = Transition(ground,st,atom,j) 
                    if trans.dipole != 0.0:
#                         print(trans.wvln)
#                         print(trans.dipole)
                        potElemSum += calcPotDetuningSum(trans,wvln)
                        scatterElemSum += calcScatterDetuningSum(trans,wvln)
#                         print(potElemSum)
        power.append(calcPower(potElemSum,wvln))
        scatter.append(calcScatterRate(wvln,scatterElemSum))

    return (power,scatter)


In [26]:
#MAIN

st = []
ElectronStates(TOTAL_STATES,st)

#Lasers
wavelengths = np.linspace(.0000002,.000001,400)
# wavelengths = np.linspace(.00000084,.00000092,50)



# +1 Polarization
#CESIUM 6S to 7P transition
cs_t = Transition(State(6,0,1/2,1/2),State(7,1,3/2,3/2),cs,None)
p_cs,polar_cs = createSpectrum(cs_t.s1,wavelengths,st,cs)

# createSpectrum(cs_transition)
#RUBIDIUM 5S to 6P transition

rb_t = Transition(State(5,0,1/2,1/2),State(6,1,3/2,3/2),rb,None)
p_rb,polar_rb = createSpectrum(rb_t.s1,wavelengths,st,rb)

/Users/jordankemp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
/Users/jordankemp/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

...

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is not triangular!
6j-Symbol is n

In [27]:
%matplotlib qt

wvlns=wavelengths*10**9
plt.plot(wvlns,np.array(p_cs)*1000)
plt.title('Power density for 20MHz Cesium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()

# plt.axvline(x=852, color="black")
plt.axhline(y=0, color="black")

plt.ylim(top=20,bottom=-50)

# print(p_cs)

plt.show()


In [28]:
# plt.plot(wvlns[130:180],np.array(polar_cs[130:180]))
plt.semilogy(wvlns,np.array(polar_cs))



plt.title('Scattering Rate of Cesium in 5mW Traps')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Scattering Rate (1/s)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()
plt.axhline(y=0, color="black")

# plt.ylim(top=3*10**-17)

# print(polar_cs)



In [29]:
wvlns=wavelengths*1000000000
plt.plot(wvlns,np.array(p_rb)*1000)
plt.title('Power density for 20MHz Rubidium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.grid()
plt.rcParams['figure.figsize'] = [15, 10]

# print(p_rb)
plt.ylim(top=20,bottom=-50)


plt.axhline(y=0, color="black")
# plt.ylim(top=1000)
# plt.ylim(top=1000,bottom=-.5)




/Users/jordankemp/miniconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:1479: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ret = ax.set_ylim(*args, **kwargs)


In [30]:
# plt.plot(wvlns[200:300],np.array(polar_rb[200:300]))
plt.semilogy(wvlns,np.array(polar_rb))
plt.title('Scattering Rate of Rubidium in 5mW Traps')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Rate (1/s)")
plt.rcParams['figure.figsize'] = [15, 10]

# plt.ylim(top=3*10**-15)



plt.grid()
plt.axhline(y=0, color="black")
# print(polar_rb)

In [38]:
%matplotlib qt

wvlns=wavelengths*1000000000
plt.plot(wvlns,np.array(p_rb)*1000)
plt.plot(wvlns,np.array(p_cs)*1000)

plt.axhline(y=0,color="black")
plt.ylim(top=25,bottom=-15)


plt.title('Power density for 20MHz Rubidium and Cesium traps at various wavelengths')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Power Density (mW/atom)")
plt.rcParams['figure.figsize'] = [15, 10]
plt.grid()




In [40]:
%matplotlib qt


plt.semilogy(wvlns,np.array(polar_rb))
plt.semilogy(wvlns,np.array(polar_cs))


plt.title('Scattering Rate of Rubidium and Cesium in 5mW Traps')
plt.xlabel("Wavelength (nm)")
plt.ylabel("Rate (1/s)")
plt.rcParams['figure.figsize'] = [15, 10]




plt.grid()
plt.axhline(y=0, color="black")
# print(polar_rb)

In [33]:
from arc import *
atom = Rubidium()
# transition 5 S_{1/2} m_j=-0.5 -> 5 P_{3/2} m_j=-1.5 for laser
# driving sigma- transition
print(atom.getDipoleMatrixElement(5,0,0.5,-0.5,5,1,1.5,-1.5,-1))

2.978


In [24]:
[938]

[938]

In [116]:
const.epsilon_0

8.854187817620389e-12